In [1]:
from ml_api import *
import pandas as pd
import numpy as  np

In [2]:
# загружаем train dataset
data = pd.read_csv('set_trn.csv')

In [3]:
data['target'] = 1
data.loc[data['store_and_fwd_flag'] == 'N', 'target'] = 0

In [4]:
# каждый инстанс модели имеет уникальный uuid сессии. Каждый новый инстанс имеет доступ к личном простарнству для 
# хранения выборок и результатов работы моделей
api = MlApi()

In [5]:
# загружаем train dataset на сервер
api.load_dataset('train',data)

data uploading...
data upload


In [6]:
# добавляем модели 
api.add_model('LG','LogisticRegression',{})
api.add_model('RF','RandomForestClassifier',{})

In [7]:
api.gel_model_list()

,Model name,type,params,fitted
0,LG,LogisticRegression,{},False
1,RF,RandomForestClassifier,{},False


In [8]:
# так можно узнать список доступных классов моделей на сервере
api.models_available()

['LogisticRegression',
 'RandomForestClassifier',
 'LinearRegression',
 'RandomForestRegressor',
 'DecisionTreeClassifier',
 'DecisionTreeRegressor',
 'KNeighborsClassifier',
 'KNeighborsRegressor']

In [9]:
# обчаем модель на факторах из short_list с целевой переменной target
short_list = ['passenger_count', 'pickup_longitude', 'pickup_latitude']
api.fit_model('LG', short_list, 'target')

Fitting LG...
Model LG fitted


In [10]:
api.gel_model_list()

,Model name,type,params,fitted
0,LG,LogisticRegression,{},True
1,RF,RandomForestClassifier,{},False


In [11]:
# подгружаем eval dataset
data = pd.read_csv('set_test.csv')
data['target'] = 1
data.loc[data['store_and_fwd_flag'] == 'N', 'target'] = 0

api.load_dataset('eval',data)

data uploading...
data upload


In [12]:
# оценим модель по мтерике accuracy и roc_auc. Для того, чтобы задать другую метрику необходимо
# в словаре metrics в качестве ключа дать название класса из https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
# а в качестве значения, что необходимо метрике: предсказание моделей(predict) или вывод вероятностей модели(predict_proba)
metrics = {
    'accuracy_score':'predict',
    'roc_auc_score' : 'predict_proba'
}
api.evaluate_model('LG', metrics)

Evaluating LG...
Model LG evaluated


In [13]:
# выводим реузльтаты оценки модели
api.get_evaluation_results()

{'LG': {'accuracy_score': 0.9945131763749852,
  'roc_auc_score': 0.5732395325586681},
 'RF': {}}

In [14]:
# так можно получить предсказания для любого сета (параметр функции sample_type)
preds = api.predict_model('LG', 'predict', 'eval')

Predicting LG...
Model LG predicted


In [15]:
preds

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
# так можно удалить модель
api.delete_model('LG')

Model LG deleted


In [17]:
api.gel_model_list()

,Model name,type,params,fitted
0,RF,RandomForestClassifier,{},False


In [18]:
api.fit_model('RF', short_list, 'target')

Fitting RF...
Model RF fitted


In [19]:
# давайте дадим новые гиперпарметры для RF 
# (!!очень важно!!: если изменить гиперпараметры, то информация об оценки модели сотрется, а также необходимо будет заного её обучить)
api.change_hyperparams('RF', params = {'n_estimators':37, 'random_state' : 42})

Hyperparameters are changed


In [20]:
api.gel_model_list()

,Model name,type,params,fitted
0,RF,RandomForestClassifier,"{'n_estimators': 37, 'random_state': 42}",False


In [21]:
# по понятным причинам у нас не получилось оценить модель, так как она еще не обучена
metrics = {
    'accuracy_score':'predict',
    'roc_auc_score' : 'predict_proba'
}
api.evaluate_model('RF', metrics)

Evaluating RF...


Exception: Model not fitted

In [22]:
api.fit_model('RF', short_list, 'target')

Fitting RF...
Model RF fitted


In [23]:
metrics = {
    'accuracy_score':'predict',
    'roc_auc_score' : 'predict_proba'
}
api.evaluate_model('RF', metrics)

Evaluating RF...
Model RF evaluated


In [24]:
api.get_evaluation_results()

{'RF': {'accuracy_score': 0.9941132621166022,
  'roc_auc_score': 0.5075025686949578}}